# Importing the database
From the course "SQL Beginner to Advanced For Data Professionals", I got the Atliq Database from 2017 to 2021. Here, I'm going to create my interest df with just the LATAM customers

In [1]:
import pymysql
from sqlalchemy import create_engine

username = 'root'
password = ''
host = 'localhost'
database = 'gdb0041'


In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',50)
pd.set_option('display.max_columns',50)
pd.options.display.float_format='{:,.2f}'.format

In [3]:

# Create a connection string
connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"
engine = create_engine(connection_string)

In [4]:
query = """
WITH CustomerLATAM AS (
    SELECT * 
    FROM dim_customer 
    WHERE region = 'LATAM'
)

SELECT 
    cl.*,
    fsm.*,
    dp.*,
    fgp.*,
    (fgp.gross_price * fsm.sold_quantity) AS total_gross_sales
FROM 
    CustomerLATAM cl
JOIN 
    fact_sales_monthly fsm ON cl.customer_code = fsm.customer_code
JOIN 
    dim_product dp ON fsm.product_code = dp.product_code
JOIN
    fact_gross_price fgp ON fsm.product_code = fgp.product_code;



"""
bad_df = pd.read_sql(query, engine)


In [5]:
bad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137424 entries, 0 to 137423
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   customer_code      137424 non-null  int64  
 1   customer           137424 non-null  object 
 2   platform           137424 non-null  object 
 3   channel            137424 non-null  object 
 4   market             137424 non-null  object 
 5   sub_zone           137424 non-null  object 
 6   region             137424 non-null  object 
 7   date               137424 non-null  object 
 8   product_code       137424 non-null  object 
 9   customer_code      137424 non-null  int64  
 10  sold_quantity      137424 non-null  int64  
 11  product_code       137424 non-null  object 
 12  division           137424 non-null  object 
 13  segment            137424 non-null  object 
 14  category           137424 non-null  object 
 15  product            137424 non-null  object 
 16  va

In [6]:
bad_df['date'].unique()

array([datetime.date(2017, 9, 1), datetime.date(2017, 10, 1),
       datetime.date(2017, 11, 1), datetime.date(2017, 12, 1),
       datetime.date(2018, 1, 1), datetime.date(2018, 2, 1),
       datetime.date(2018, 3, 1), datetime.date(2018, 4, 1),
       datetime.date(2018, 5, 1), datetime.date(2018, 6, 1),
       datetime.date(2018, 7, 1), datetime.date(2018, 8, 1),
       datetime.date(2018, 9, 1), datetime.date(2018, 10, 1),
       datetime.date(2018, 11, 1), datetime.date(2018, 12, 1),
       datetime.date(2019, 1, 1), datetime.date(2019, 2, 1),
       datetime.date(2019, 3, 1), datetime.date(2019, 4, 1),
       datetime.date(2019, 5, 1), datetime.date(2019, 6, 1),
       datetime.date(2019, 7, 1), datetime.date(2019, 8, 1),
       datetime.date(2019, 9, 1), datetime.date(2019, 10, 1),
       datetime.date(2019, 11, 1), datetime.date(2019, 12, 1),
       datetime.date(2020, 1, 1), datetime.date(2020, 2, 1),
       datetime.date(2020, 3, 1), datetime.date(2020, 4, 1),
       datetime

In [7]:
bad_df.head()

,customer_code,customer,platform,channel,market,sub_zone,region,date,product_code,customer_code,sold_quantity,product_code,division,segment,category,product,variant,product_code,fiscal_year,gross_price,total_gross_sales
0,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-09-01,A0118150101,90027207,4,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,61.58
1,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-09-01,A0118150101,90027207,4,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2019,14.44,57.76
2,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-09-01,A0118150101,90027207,4,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2020,16.23,64.93
3,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-09-01,A0118150101,90027207,4,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2021,19.06,76.23
4,70027208,Atliq e Store,E-Commerce,Direct,Brazil,LATAM,LATAM,2017-10-01,A0118150101,70027208,5,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,76.98


Okey, I've understood the problem here. In our RDBMS, we have different price for each fiscal year:

In [8]:
query2 = """
SELECT *
FROM fact_gross_price
"""
fact_gross_price = pd.read_sql(query2, engine)
fact_gross_price.head()

,product_code,fiscal_year,gross_price
0,A0118150101,2018,15.40
1,A0118150101,2019,14.44
2,A0118150101,2020,16.23
3,A0118150101,2021,19.06
4,A0118150102,2018,19.59


As we are joining the tables by the product code and it is multiplying the total_gross_sales of every product by each fiscal year, ye don't need all these extra rows, because we just want the ones that are the same date with the fiscal year. It starts from September 2017, so what we are going to do??

In [9]:
# I just asked the query to ChatGPT4 - Code Interpreter tbh
query3 = """
WITH CustomerLATAM AS (
    SELECT * 
    FROM dim_customer 
    WHERE region = 'LATAM'
)

SELECT 
    cl.*,
    fsm.*,
    dp.*,
    fgp.*,
    (fgp.gross_price * fsm.sold_quantity) AS total_gross_sales
FROM 
    CustomerLATAM cl
JOIN 
    fact_sales_monthly fsm ON cl.customer_code = fsm.customer_code
JOIN 
    dim_product dp ON fsm.product_code = dp.product_code
JOIN
    fact_gross_price fgp ON fsm.product_code = fgp.product_code
WHERE 
    (MONTH(fsm.date) >= 9 AND YEAR(fsm.date) + 1 = fgp.fiscal_year)
    OR
    (MONTH(fsm.date) < 9 AND YEAR(fsm.date) = fgp.fiscal_year);

"""
df = pd.read_sql(query3, engine)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39294 entries, 0 to 39293
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_code      39294 non-null  int64  
 1   customer           39294 non-null  object 
 2   platform           39294 non-null  object 
 3   channel            39294 non-null  object 
 4   market             39294 non-null  object 
 5   sub_zone           39294 non-null  object 
 6   region             39294 non-null  object 
 7   date               39294 non-null  object 
 8   product_code       39294 non-null  object 
 9   customer_code      39294 non-null  int64  
 10  sold_quantity      39294 non-null  int64  
 11  product_code       39294 non-null  object 
 12  division           39294 non-null  object 
 13  segment            39294 non-null  object 
 14  category           39294 non-null  object 
 15  product            39294 non-null  object 
 16  variant            392

In [11]:
df.groupby('fiscal_year')['total_gross_sales'].describe()

,count,mean,std,min,25%,50%,75%,max
fiscal_year,,,,,,,,
2018,"1,076.00",366.50,392.04,16.33,153.95,290.46,435.10,"5,887.34"
2019,"4,227.00",360.11,427.75,14.06,145.54,240.49,415.86,"5,352.76"
2020,"9,407.00",466.64,575.04,2.92,75.45,284.88,609.45,"6,218.24"
2021,"17,344.00",361.95,307.95,8.00,127.13,286.90,521.26,"2,646.58"
2022,"7,240.00","1,724.58","1,852.88",16.88,519.03,"1,119.18","2,233.08","17,427.59"


In [12]:
df

,customer_code,customer,platform,channel,market,sub_zone,region,date,product_code,customer_code,sold_quantity,product_code,division,segment,category,product,variant,product_code,fiscal_year,gross_price,total_gross_sales
0,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-09-01,A0118150101,90027207,4,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,61.58
1,70027208,Atliq e Store,E-Commerce,Direct,Brazil,LATAM,LATAM,2017-10-01,A0118150101,70027208,5,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,76.98
2,70027208,Atliq e Store,E-Commerce,Direct,Brazil,LATAM,LATAM,2017-11-01,A0118150101,70027208,5,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,76.98
3,90027207,Amazon,E-Commerce,Retailer,Brazil,LATAM,LATAM,2017-11-01,A0118150101,90027207,16,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,246.32
4,70027208,Atliq e Store,E-Commerce,Direct,Brazil,LATAM,LATAM,2017-12-01,A0118150101,70027208,3,A0118150101,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,A0118150101,2018,15.40,46.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39289,70026206,Atliq e Store,E-Commerce,Direct,Mexico,LATAM,LATAM,2021-12-01,A7321160303,70026206,36,A7321160303,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,A7321160303,2022,43.56,"1,568.01"
39290,70027208,Atliq e Store,E-Commerce,Direct,Brazil,LATAM,LATAM,2021-12-01,A7321160303,70027208,28,A7321160303,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,A7321160303,2022,43.56,"1,219.57"
39291,90024183,Electricalsbea Stores,Brick & Mortar,Retailer,Chile,LATAM,LATAM,2021-12-01,A7321160303,90024183,25,A7321160303,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,A7321160303,2022,43.56,"1,088.90"
39292,90024184,Amazon,E-Commerce,Retailer,Chile,LATAM,LATAM,2021-12-01,A7321160303,90024184,25,A7321160303,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,A7321160303,2022,43.56,"1,088.90"


Let's drop the unuseful columns to export our df.  
- don't need region, because all customers are from LATAM
- don't need 'customer_code', and 'product_code', because their purpose is for the RDBMS. Now we have retrieved the dataset.
- don't need the fiscal year, neither the gross price by each product

In [13]:
df2 = df.drop(['sub_zone','region','customer_code','product_code','gross_price','fiscal_year'], axis='columns')

In [14]:
df2

,customer,platform,channel,market,date,sold_quantity,division,segment,category,product,variant,total_gross_sales
0,Amazon,E-Commerce,Retailer,Brazil,2017-09-01,4,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,61.58
1,Atliq e Store,E-Commerce,Direct,Brazil,2017-10-01,5,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,76.98
2,Atliq e Store,E-Commerce,Direct,Brazil,2017-11-01,5,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,76.98
3,Amazon,E-Commerce,Retailer,Brazil,2017-11-01,16,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,246.32
4,Atliq e Store,E-Commerce,Direct,Brazil,2017-12-01,3,P & A,Peripherals,Internal HDD,AQ Dracula HDD – 3.5 Inch SATA 6 Gb/s 5400 RPM...,Standard,46.19
...,...,...,...,...,...,...,...,...,...,...,...,...
39289,Atliq e Store,E-Commerce,Direct,Mexico,2021-12-01,36,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,568.01"
39290,Atliq e Store,E-Commerce,Direct,Brazil,2021-12-01,28,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,219.57"
39291,Electricalsbea Stores,Brick & Mortar,Retailer,Chile,2021-12-01,25,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,088.90"
39292,Amazon,E-Commerce,Retailer,Chile,2021-12-01,25,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,088.90"


In [15]:
df2['market'].value_counts()

market
Brazil      14021
Mexico      12736
Chile        8755
Columbia     3782
Name: count, dtype: int64

In [16]:
df2['platform'].value_counts()

platform
E-Commerce        31225
Brick & Mortar     8069
Name: count, dtype: int64

In [17]:
df2['channel'].value_counts()

channel
Retailer    25853
Direct      13441
Name: count, dtype: int64

In [18]:
df2['segment'].value_counts()

segment
Accessories    13700
Peripherals    10710
Notebook       10335
Storage         2415
Networking      1221
Desktop          913
Name: count, dtype: int64

In [19]:
df2['category'].value_counts()

category
Keyboard                       6144
Mouse                          5900
Personal Laptop                5575
Graphic Card                   4460
Business Laptop                3584
Processors                     2513
MotherBoard                    2221
External Solid State Drives    1954
Gaming Laptop                  1795
Batteries                      1656
Internal HDD                   1516
Wi fi extender                 1221
USB Flash Drives                461
Personal Desktop                294
Name: count, dtype: int64

In [20]:
df2.tail(10)

,customer,platform,channel,market,date,sold_quantity,division,segment,category,product,variant,total_gross_sales
39284,Electricalsbea Stores,Brick & Mortar,Retailer,Chile,2021-11-01,35,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,524.46"
39285,Amazon,E-Commerce,Retailer,Chile,2021-11-01,43,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,872.90"
39286,Electricalsbea Stores,Brick & Mortar,Retailer,Columbia,2021-11-01,18,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,784.01
39287,Amazon,E-Commerce,Retailer,Mexico,2021-11-01,48,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"2,090.68"
39288,Amazon,E-Commerce,Retailer,Brazil,2021-11-01,55,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"2,395.57"
39289,Atliq e Store,E-Commerce,Direct,Mexico,2021-12-01,36,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,568.01"
39290,Atliq e Store,E-Commerce,Direct,Brazil,2021-12-01,28,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,219.57"
39291,Electricalsbea Stores,Brick & Mortar,Retailer,Chile,2021-12-01,25,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,088.90"
39292,Amazon,E-Commerce,Retailer,Chile,2021-12-01,25,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,088.90"
39293,Amazon,E-Commerce,Retailer,Brazil,2021-12-01,23,N & S,Networking,Wi fi extender,AQ Wi Power Dx3,Premium,"1,001.79"


In [21]:
df2['customer'].value_counts()

customer
Amazon                   17784
Atliq e Store            13441
Electricalsbea Stores     8069
Name: count, dtype: int64

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39294 entries, 0 to 39293
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer           39294 non-null  object 
 1   platform           39294 non-null  object 
 2   channel            39294 non-null  object 
 3   market             39294 non-null  object 
 4   date               39294 non-null  object 
 5   sold_quantity      39294 non-null  int64  
 6   division           39294 non-null  object 
 7   segment            39294 non-null  object 
 8   category           39294 non-null  object 
 9   product            39294 non-null  object 
 10  variant            39294 non-null  object 
 11  total_gross_sales  39294 non-null  float64
dtypes: float64(1), int64(1), object(10)
memory usage: 3.6+ MB


In [23]:
df2['product'].nunique()

71

Wow, this data is amazing! The number of things we can do and visualize is huge. Let's export our data into a pickle file n.n

In [26]:
pd.to_pickle(df2, '../../data/raw/AtliQ_sales_LATAM.pkl')

In [25]:
df2.to_csv('../../data/raw/AtliQ_sales_LATAM.csv', index=False)